In [10]:
import matplotlib.pyplot as plt
from tensorflow import keras

In [11]:
fashoin_mnist = keras.datasets.fashion_mnist

In [12]:
(train_images,train_labels),(test_images,test_labels) = fashoin_mnist.load_data()

In [13]:
train_images = train_images/255.0
test_images = test_images/255.0

In [14]:
train_images[0].shape #2d image

(28, 28)

In [15]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [16]:
train_images[0].shape #3d image

(28, 28, 1)

In [17]:
# Building a model
def build_model(hp):
    model = keras.Sequential([
    # first convolutional layer
    keras.layers.Conv2D(
        filters = hp.Int("conv_1_filter", min_value=32, max_value=128, step=16), # number of filters in a kernel
        kernel_size = hp.Choice("conv_1_kernel", values=[3,5]), #size of a filter
        activation = "relu",
        input_shape = (28,28,1)),
    
    # Second convolutional layer
    keras.layers.Conv2D(
        filters = hp.Int("conv_2_filter",min_value=32, max_value=64, step=16),
        kernel_size = hp.Choice("conv_1_kernel", values=[3,5]), #size of a filter
        activation = "relu"),
    
    # Flatter and fully connected layer 
    keras.layers.Flatten(),
    keras.layers.Dense(
        units= hp.Int("dense_1_units", min_value=32, max_value=128, step=16),
        activation="relu"),
    
    # output layer
    keras.layers.Dense(10,activation="softmax")          
                             ])
    # Compiling the model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", values=[1e-2,1e-3])),
                 loss="sparse_categorical_crossentropy",
                 metrics=["accuracy"])
    return model

In [18]:
!pip install keras-tuner
from kerastuner import RandomSearch, HyperParameters
tuner_search= RandomSearch(build_model,
                           objective="val_accuracy",
                           max_trials=5, 
                           directory="output", project_name="MNIST Fashion")

     |████████████████████████████████| 71kB 6.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78935 sha256=5d31838ca7a2afbc99bdf35d3b8b676403f626c7c1e1bf0f95fedd8a7c372cfb
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=df7bc379d2ece182ca0299a5f41b1c1322d7eacb3123ea2ec29654b8b4094c96
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [19]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 21s]
val_accuracy: 0.9103333353996277

Best val_accuracy So Far: 0.9103333353996277
Total elapsed time: 00h 03m 08s
INFO:tensorflow:Oracle triggered exit


In [23]:
model = tuner_search.get_best_models(num_models=1)[0]

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                3539040   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,577,578
Trainable params: 3,577,578
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1113 - accuracy: 0.9591 - val_loss: 0.2627 - val_accuracy: 0.9132
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0746 - accuracy: 0.9722 - val_loss: 0.2816 - val_accuracy: 0.9158
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0482 - accuracy: 0.9826 - val_loss: 0.3419 - val_accuracy: 0.9097
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0369 - accuracy: 0.9865 - val_loss: 0.3846 - val_accuracy: 0.9210
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0262 - accuracy: 0.9909 - val_loss: 0.4641 - val_accuracy: 0.9168
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0218 - accuracy: 0.9924 - val_loss: 0.4650 - val_accuracy: 0.9183
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0207 - accuracy: 0.9931 - val_loss: 0.5385 - val_accuracy